In [1]:
import requests 
import datetime
import pandas as pd
import altair as alt
import json


from dotenv import load_dotenv
import os

load_dotenv()

# api_key=os.getenv("WEATHERAPI_APIKEY")

True

In [2]:
# Historical data does not work with the free key
# response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={api_key}&q=Berlin&dt=2015-01-01")
# berlin = response.json()

In [3]:
# I thought about doing an API-Call, but downloading the csv was just so much easier. 

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# berlin = response.json()

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=40.6782&longitude=73.9442&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# new_york = response.json()

Data source: https://open-meteo.com/

This describtion works for both dataframes - one will have berlin added, the other one will have brooklyn added

day: date of the measures
temperature: mean temperature on that day in Celcius
rain: sum of rain on that day in millimeter
snowfall: sum of snowfall on that day in Centimeter
sunshine: duration of sunshine in that day in hours (converted von seconds to hours)

In [4]:
df = pd.read_csv('raw/weather_brooklyn.csv', header=2)

In [5]:
# rename columns

(
    df
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine',
            'weather_code (wmo code)' : 'weather_code',
            'relative_humidity_2m_mean (%)' : 'humidity',
            'apparent_temperature_mean (°C)' : 'feels_like',
            'daylight_duration (s)' : 'daylight_duration'
        }, 
        inplace=True
    )
)

In [6]:
df

,day,weather_code,humidity,temperature,feels_like,daylight_duration,sunshine,rain,snowfall
0,2015-07-15,1,49,18.4,17.2,53293.71,49436.16,0.0,0.0
1,2015-07-16,2,54,18.3,17.6,53211.77,49392.71,0.0,0.0
2,2015-07-17,61,73,16.2,16.3,53127.41,47537.50,2.8,0.0
3,2015-07-18,61,81,14.8,14.9,53040.73,42230.91,3.4,0.0
4,2015-07-19,53,82,13.7,13.2,52951.55,37390.75,3.3,0.0
...,...,...,...,...,...,...,...,...,...
3619,2025-06-11,51,89,8.1,6.6,54153.83,30466.62,1.4,0.0
3620,2025-06-12,63,90,9.6,8.7,54187.14,19564.03,14.3,0.0
3621,2025-06-13,53,87,9.2,8.3,54216.68,46800.00,3.1,0.0
3622,2025-06-14,53,91,7.8,6.5,54242.38,10124.81,4.5,0.0


In [7]:
# calculate sunshine from seconds to hours

df['sunshine_hours'] = df['sunshine'] / 3600
df = df.drop('sunshine', axis=1)

In [8]:
# calculate daylight from seconds to hours

df['daylight_hours'] = df['daylight_duration'] / 3600
df = df.drop('daylight_duration', axis=1)

In [9]:
# add month to data
df['day'] = pd.to_datetime(df['day'])

In [10]:
df['month_year'] = df['day'].dt.to_period('M')
df['year'] = df['day'].dt.to_period('Y')

In [11]:
# add season according to month
# Dez, Jan, Feb == Winter
# Mar, Apr, Mai == Spring
# Jun, Jul, Aug == Summer
# Sep, Oct, Nov == Fall

df['month_num'] = df['day'].dt.month

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'

df['season'] = df['month_num'].apply(get_season)

In [12]:
# match weathercodes with description




In [13]:
df.dtypes

day               datetime64[ns]
weather_code               int64
humidity                   int64
temperature              float64
feels_like               float64
rain                     float64
snowfall                 float64
sunshine_hours           float64
daylight_hours           float64
month_year             period[M]
year               period[Y-DEC]
month_num                  int32
season                    object
dtype: object

In [14]:
df['day_of_week'] = df['day'].dt.day_name()

In [15]:
df

,day,weather_code,humidity,temperature,feels_like,rain,snowfall,sunshine_hours,daylight_hours,month_year,year,month_num,season,day_of_week
0,2015-07-15,1,49,18.4,17.2,0.0,0.0,13.732267,14.803808,2015-07,2015,7,summer,Wednesday
1,2015-07-16,2,54,18.3,17.6,0.0,0.0,13.720197,14.781047,2015-07,2015,7,summer,Thursday
2,2015-07-17,61,73,16.2,16.3,2.8,0.0,13.204861,14.757614,2015-07,2015,7,summer,Friday
3,2015-07-18,61,81,14.8,14.9,3.4,0.0,11.730808,14.733536,2015-07,2015,7,summer,Saturday
4,2015-07-19,53,82,13.7,13.2,3.3,0.0,10.386319,14.708764,2015-07,2015,7,summer,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,2025-06-11,51,89,8.1,6.6,1.4,0.0,8.462950,15.042731,2025-06,2025,6,summer,Wednesday
3620,2025-06-12,63,90,9.6,8.7,14.3,0.0,5.434453,15.051983,2025-06,2025,6,summer,Thursday
3621,2025-06-13,53,87,9.2,8.3,3.1,0.0,13.000000,15.060189,2025-06,2025,6,summer,Friday
3622,2025-06-14,53,91,7.8,6.5,4.5,0.0,2.812447,15.067328,2025-06,2025,6,summer,Saturday


In [16]:
with open("raw/descriptions.json") as f:
    weather_dict = json.load(f)

In [17]:
weather_dict

{'0': {'day': {'description': 'Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '1': {'day': {'description': 'Mainly Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Mainly Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '2': {'day': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02d@2x.png'},
  'night': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02n@2x.png'}},
 '3': {'day': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03d@2x.png'},
  'night': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03n@2x.png'}},
 '45': {'day': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50d@2x.png'},
  'night': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50

In [18]:
df['weather_code'] = df['weather_code'].astype(str)

In [19]:
df['weather_description'] = df['weather_code'].map( lambda x: weather_dict.get(x, {}).get('day', {}).get('description', 'Unknown'))
df['weather_description']

0        Mainly Sunny
1       Partly Cloudy
2          Light Rain
3          Light Rain
4             Drizzle
            ...      
3619    Light Drizzle
3620             Rain
3621          Drizzle
3622          Drizzle
3623             Rain
Name: weather_description, Length: 3624, dtype: object

In [20]:
# save cleaned file with calculations

df.to_csv('processed/brooklyn_cleaned.csv', index=False)